# Tooling

## Testing

With a growing code base and increasing complexity of software projects, manual or exploratory testing becomes unfeasible. Automated tests allow the programmer to write tests for their application by formulating assumptions, a.k.a. __test cases__, about their code explicitly and checking them automatically. A __test runner__ sets up the test environment, detects test cases, and runs these cases in the specified environment. In genereal, testing is divided into unit testing, integration testing, and system testing. With __unit tests__ the smallest components of a software are tested. In the case of Python, unit tests are written for functions and modules. Integration tests are conducted with multiple components at once, e.g. several modules, while systems tests are reserved for the whole system.

___References___
* [UnitTest Documentation](https://docs.python.org/3/library/unittest.html)
* [PyTest Documentation](https://docs.pytest.org/en/latest/)

In [1]:
%%writefile tooling/testing/fibonacci.py

def fibonacci(n):
    if not isinstance(n, int):
        raise TypeError("Parameter \'n\' must be an integer")
        
    if (n < 0):
        return []
    elif (n == 0):
        return [0]
    elif (n == 1):
        return [0, 1]
    else:
        fibs = [0, 1]
        for i in range(1, n):
            s = fibs[-2] + fibs[-1]
            fibs.append(s)
        return fibs

Overwriting tooling/testing/fibonacci.py


#### UnitTest

`unittest` is part of the Python standard library and contains a testing framework as well as a test runner. To write a unit test, it requires the tester to write tests as methods in classes and to use a set of specific assertion methods, such as `assertIsNot(a, b)` or `assertIsInstance(a, b)`. The test environment is generated by the methods `setUp()` and `tearDown()` that are executed before and after each test case. Alternatively, `setUpClass()` and `tearDownClass()` are executed only once for the whole class.

In [2]:
%%writefile tooling/testing/test_fibonacci.py

import unittest
from fibonacci import fibonacci


class TestFibonacci(unittest.TestCase):
    
    @classmethod
    def setUpClass(self):
        print("Setting up test class environment ...")
    
    
    @classmethod
    def tearDownClass(self):
        print("Tearing down test class environment ...")
    
    
    def setUp(self):
        print("Setting up test case environment ...")
        
        
    def tearDown(self):
        print("Tearing down test case environment ...")

        
    def test_int_case(self):
        with self.assertRaises(TypeError):
            fibonacci('x')

    
    def test_base_case0(self):
        fib0 = fibonacci(0)
        self.assertIsInstance(fib0, list, "fibonacci(0) should be a list")
        self.assertEqual(len(fib0), 1, "fibonacci(0) should have length 1")
        self.assertEqual(fib0[0], 0, "fibonacci(0) at index 0 should be 0")

    
    def test_base_case1(self):
        fib1 = fibonacci(1)
        self.assertIsInstance(fib1, list, "fibonacci(1) should be a list")
        self.assertEqual(len(fib1), 2, "fibonacci(1) should have length 2")
        self.assertEqual(fib1[1], 1, "fibonacci(1) at index 1 should be 1")


    def test_first_case(self):
        fib2 = fibonacci(2)
        self.assertIsInstance(fib2, list, "fibonacci(2) should be a list")
        self.assertEqual(len(fib2), 3, "fibonacci(2) should have length 3")
        self.assertEqual(fib2[2], 1, "fibonacci(2) at index 2 should be 1")

    
    def test_negative_case(self):
        fib1n = fibonacci(-1)
        self.assertIsInstance(fib1n, list, "fibonacci(-1) should be a list")
        self.assertNotEqual(len(fib1n), 0, "fibonacci(-1) should have length 0")

        
if __name__ == '__main__':
    unittest.main(verbosity=2)

Overwriting tooling/testing/test_fibonacci.py


In [3]:
!python3 tooling/testing/test_fibonacci.py

Setting up test class environment ...
test_base_case0 (__main__.TestFibonacci) ... Setting up test case environment ...
Tearing down test case environment ...
ok
test_base_case1 (__main__.TestFibonacci) ... Setting up test case environment ...
Tearing down test case environment ...
ok
test_first_case (__main__.TestFibonacci) ... Setting up test case environment ...
Tearing down test case environment ...
ok
test_int_case (__main__.TestFibonacci) ... Setting up test case environment ...
Tearing down test case environment ...
ok
test_negative_case (__main__.TestFibonacci) ... Setting up test case environment ...
Tearing down test case environment ...
FAIL
Tearing down test class environment ...

FAIL: test_negative_case (__main__.TestFibonacci)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/fs70824/trainee19/python4hpc/tooling/testing/test_fibonacci.py", line 55, in test_negative_case
    self.assertNotEqual(len(fib1

### Pytest

Unfortunately, writing tests with `unittest` requires a large amount of boilerplate code and the test runner's output becomes increasingly difficult to read with a growing number of test cases. As an alternative, `pytest` alleviates these issues with __simplified test code__ and a __comprehensible test runner output__. Additionally, it supports `unittest` test cases out of the box. Instead of several `assertThisOrThat(x)` methods, Python's built-in `assert` statement is used to formulate test cases that look much more like source code than unit test cases. The purpose of setting up the stage for tests to be executed is fulfilled by __fixtures__.

In [4]:
%%writefile tooling/testing/test_fibonacci.py

import pytest
import random
from fibonacci import fibonacci


@pytest.fixture
def setup_basic():
    print("Performing basic test setup ...")


@pytest.fixture
def setup_advanced(setup_basic):
    print("Performing advanced test setup ...")


@pytest.fixture
def create_number():
    print("Performing setup to create random number ...")
    rnd = random.randint(3, 30)
    yield rnd
    print("Performing teardown ...") 
    

def test_int_case(create_number):
    n = create_number
    fibs = fibonacci(n)
    print(fibs)
    assert len(fibs) == n+1
    
    
def test_char_case():
    with pytest.raises(TypeError):
        fibonacci('x')


def test_base_case0(setup_basic):
    fib0 = fibonacci(0)
    assert isinstance(fib0, list)
    assert len(fib0) == 1
    assert fib0[0] == 0

    
def test_base_case1(setup_basic):
    fib1 = fibonacci(1)
    assert isinstance(fib1, list)
    assert len(fib1) == 2
    assert fib1[1] == 1
    
    
def test_first_case(setup_advanced):
    fib2 = fibonacci(2)
    assert isinstance(fib2, list)
    assert len(fib2) == 3
    assert fib2[2] == 1


def test_negative_case():
    fib1n = fibonacci(-1)
    assert isinstance(fib1n, list)
    assert len(fib1n) == 0

Overwriting tooling/testing/test_fibonacci.py


In [5]:
!python3 -m pytest --capture=no tooling/testing

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/fs70824/trainee19/python4hpc
plugins: anyio-3.6.1
collected 6 items                                                              

tooling/testing/test_fibonacci.py Performing setup to create random number ...
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040]
.Performing teardown ...
.Performing basic test setup ...
.Performing basic test setup ...
.Performing basic test setup ...
Performing advanced test setup ...
..

============================== 6 passed in 0.11s ===============================


## Debugging

Python comes equipped with a full-fledged, interactive debugger that allows setting breakpoints, stepping through the code line-wise, inspecting the stack, evaluating arbitrary code in any context, and more. The Python Debugger `pdb` is part of the standard library and works similarly to the GNU debugger `gdb`. JupyterLab also provides an extension for `pdb` but it offers few features. Essentially, the extension can only step through the code and inspect the stack frames. In order to use of `pdb` in its full glory, it has to be called either from the terminal or from a development environment.

Some common commands used while debugging are the following:
* `p [expression]` evaluate the given expression in the current context
* `c` continues execution and only stop at a breakpoint
* `s` executes the current line and stop at the first possible occasion
* `n` continues execution until the next line in the current function is reached or it returns
* `r` continues execution until the current function returns
* `!` executes the given one-line statement in the current context
* `b [linenumber | function]` sets a breakpoint a the given line number òr at the given function
* `u [count]` moves to an older stack frame
* `d [count]`moves to a newer stack frame

___References___
* [The Python Debugger - Official Documentation](https://docs.python.org/3/library/pdb.html)

In [6]:
%%writefile tooling/debugging/test_fibonacci.py

def fibonacci(n):
    if (n < 0):
        return 0
    elif (n < 2):
        return n
    else:
        import pdb; pdb.set_trace()
        fibs = [0, 1]
        for i in range(2, n):
            s = fibs[i-2] + fibs[i-1]
            fibs.append(s)
        return fibs

    
fibonacci(20)

Writing tooling/debugging/test_fibonacci.py


## Documentation

The basis for the documentation of Python projects is formed by __docstrings__. An object's docstring can be viewed either by calling its `help` function or printing its `__doc__` attribute. To document a class or a function correctly, a string with triple-double quotes is placed directly under its signature.

There are several different docstring formats in use, such as [Epytext](http://epydoc.sourceforge.net/manual-epytext.html) and [Google Docstrings](https://google.github.io/styleguide/pyguide.html), however, it is highly recommended to use [reStructuredText](https://www.sphinx-doc.org/en/master/usage/restructuredtext/index.html), since it is supported by [Sphinx](https://www.sphinx-doc.org/en/master/index.html), the most common documentation generator, and it is also the format used by Python's official documentation.

In [7]:
help(int)

Help on class int in module builtins:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Built-in subclasses:
 |      bool
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      True if 

In [8]:
print(int.__doc__)

int([x]) -> integer
int(x, base=10) -> integer

Convert a number or string to an integer, or return 0 if no arguments
are given.  If x is a number, return x.__int__().  For floating point
numbers, this truncates towards zero.

If x is not a number or if base is given, then x must be a string,
bytes, or bytearray instance representing an integer literal in the
given base.  The literal can be preceded by '+' or '-' and be surrounded
by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
Base 0 means to interpret the base from the string as an integer literal.
>>> int('0b100', base=0)
4


### pydoc

`pydoc` is simply a module that automatically generates documentation from Python modules. The The documentation can be presented as pages of text on the console, served to a web browser, or saved to HTML files. In some sense, it can be seen as the `man` command from te Unix world.

In [9]:
!python3 -m pydoc int

Help on class int in module builtins:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Built-in subclasses:
 |      bool
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      True if 

### Sphinx

[Sphinx](https://www.sphinx-doc.org/en/master/index.html) is by far the most popular Python documentation tool. Even the [official Python documentation](https://docs.python.org/3/) is created with it.

In [10]:
def sphinx_docstrings(num1, num2):
    """Add up two integer numbers.

    This function simply wraps the ``+`` operator, and does not
    do anything interesting, except for illustrating what
    the docstring of a very simple function looks like.

    :param int num1: First number to add.
    :param int num2: Second number to add.
    :returns:  The sum of ``num1`` and ``num2``.
    :rtype: int
    :raises AnyError: If anything bad happens.
    """
    return num1 + num2

In [11]:
help(sphinx_docstrings)

Help on function sphinx_docstrings in module __main__:

sphinx_docstrings(num1, num2)
    Add up two integer numbers.
    
    This function simply wraps the ``+`` operator, and does not
    do anything interesting, except for illustrating what
    the docstring of a very simple function looks like.
    
    :param int num1: First number to add.
    :param int num2: Second number to add.
    :returns:  The sum of ``num1`` and ``num2``.
    :rtype: int
    :raises AnyError: If anything bad happens.



### Doxygen

[Doxygen](https://doxygen.nl/index.html) claims to be de-facto standard tool for generating documentation from annotated source code, but it is predominantly used in C or C++ projects. For projects with mixed source code, Doxygen might be a feasible option. However, it is definitely not the frst choice. In general, it is quite similar to Sphinx: The source code is annoteted with comments in a special syntax, which is subsequently used to generate HTML. It can also generate documentation from uncommented source code based on signatures.

## Logging

Logging helps to understand the flow of a program by either recording events that occur during the execution of a program, or messages between between software components. Moreover, these records are extraordinarily helpful while debugging a program or analysing the performance of a system.
The Python standard library comes equipped with a powerful `logging` module so that logging can be easily integrated into an application.
Depending on the severity of the error or the importance of the recorded event or message, the following five `logging levels` are commonly used, in order of increasing severity/importance: DEBUG, INFO, WARNING, ERROR, CRITICAL.

By importing the `logging` module, __root logger__ is immediately useable and ready to print custom messages on the terminal. However, using it like this is not much better than simply using `print` for each event.

In [ ]:
import logging


logging.debug('DEBUG message')
logging.info('INFO message')
logging.warning('WARNING message')
logging.error('ERROR message')
logging.critical('CRITICAL message')

The logger can also be [configured](https://docs.python.org/3/library/logging.html#logging.basicConfig) with parameters by calling `basicConfig`. Consider that the root logger can only be configured once, i.e. this function must not be called more then once. 

Commonly used parameters for configurations are:
* `level`: minimal severity level.
* `filename`: file that shall contain logging messgaes
* `filemode`: file opening mode (default: append)
* `format`: log message format

In [ ]:
import logging


logging.basicConfig(filename='tooling/logging/mylogfile.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')
logging.error('This will get logged to a file')

In [ ]:
import logging

a = 5
b = 0

try:
    c = a / b
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

It is recommended to set the switch `exc_info` to `True`, otherwise the output of the above program would not tell us anything about the exception.

__Handlers & Formatters__

The purpose of [handlers](https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers) is to send the log records to the appropriate destination. Several types of handlers, such as StreamHandler and FileHandler, are available albeit logs are typically written to a log file.
[Formatters](https://docs.python.org/3/library/logging.html#formatter-objects) format the output according to the given specification, which is especially useful for serve logs in different formats to several handlers, e.g. shorter log messages for the terminal and more detailed ones for log files.

In [ ]:
import logging


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename='tooling/logging/myapp.log',
                    filemode='w')

console = logging.StreamHandler()
console.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

logging.info('Jackdaws love my big sphinx of quartz.')

logger1 = logging.getLogger('myapp.area1')
logger2 = logging.getLogger('myapp.area2')

logger1.debug('Quick zephyrs blow, vexing daft Jim.')
logger1.info('How quickly daft jumping zebras vex.')
logger2.warning('Jail zesty vixen who grabbed pay from quack.')
logger2.error('The five boxing wizards jump quickly.')